In [0]:
import logging
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType, LongType 
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
import requests
from datetime import datetime
from pyspark.sql.functions import when, col, round, max

In [0]:
logger = logging.getLogger("DLTLoggerGoldNintendo")

logger.setLevel(logging.INFO)

if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('[%(levelname)s] %(asctime)s - %(message)s',
                                  '%Y-%m-%d %H:%M:%S')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

In [0]:
def get_cdi_today():
    url = "https://brasilapi.com.br/api/taxas/v1/cdi"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        cdi = 1 + (data['valor'] / 100)
        return cdi
    except requests.exceptions.RequestException as e:
        print(f"Erro ao conectar à API da Brasil API: {e}")
        return None
    except (ValueError, IndexError) as e:
        print(f"Erro ao processar os dados da API: {e}")
        return None

In [0]:
@dlt.table(
    name="nintendodatabricks_workspace.gold.consoles_recomendation",
    comment="Tabela de recomendação de consoles particionada por liquid cluster",
    cluster_by=["codigo","extract"]
)

@dlt.expect("recomendacao_string_not_null", "typeof(recomendacao) = 'string' AND recomendacao IS NOT NULL")

def console_gold():

    logger.info("Leitura da tabela Silver iniciada...")

    cdi_data = get_cdi_today()

    df = dlt.read("nintendodatabricks_workspace.silver.consoles_silver") 

    df_price_desconto = df.withColumn("preco_desconto", round(col("preco") - (col("preco") * col("desconto")), 2))

    df_price_parcelado = df_price_desconto.withColumn("preco_parcelado", round((col("valor_prestacao") * col("numero_parcelas")), 2))

    df_save_desconto = df_price_parcelado.withColumn("save_desconto", round((col("preco") - col("preco_desconto")), 2))

    df_save_parcelado = df_save_desconto.withColumn(
        "save_parcelado",
        when((col("preco") - col("preco_parcelado")) * cdi_data <= 0, 0)
        .otherwise(round(((col("preco") - col("preco_parcelado")) * cdi_data), 2))
    )

    df_pct_parcelado = df_save_parcelado.withColumn(
        "pct_preco_parcelado",
        round(100 -((col("preco_parcelado") / col("preco")) * 100), 2)
    )

    df_final = df_pct_parcelado.withColumn(
        "recomendacao",
        when(col("desconto") < col("pct_preco_parcelado"), "Comprar Parcelado")
        .when(col("desconto") > col("pct_preco_parcelado"), "Comprar a Vista Pix")
        .otherwise("Sem Vantagem Identificada")
    ).drop("preco_desconto", "preco_parcelado", "save_desconto", "save_parcelado", "pct_preco_parcelado")

    logger.info("Carregando tabela de recomendação")

    return df_final